In [5]:
import hashlib
import random
import string
import time
from collections import deque
import sys
from math import ulp

In [27]:
class Entity:
    def __init__(self, id):

        self.id = id
        self.entityTransactions = []

    # step1 : Calculating hash (finding the nonce that satisfies the defined target)
    # step2 : Selecting two transactions in a way
    # step3 : Check if two transactions are not conflicting
    # step4 : Apply required changes in variables
    # step5 : Update cumulative weights and check for confirmed transactions

    def send_transaction(self):
        timestamp = time.time()
        tx_id = random.randint(0, 100)
        transaction = Transaction(tx_id, timestamp, self.id)

        # step1
        transaction.hash = transaction.calculate_hash()
        print('Transaction hash: ', transaction.hash)

        # step2
        selected_transactions = self.transaction_selection(tipsList, unconfirmedList, 2)
        print("\nSelected transactions' IDs:")
        for tx in selected_transactions:
            print(tx.id)

        # step3
        self.validate_transactions(selected_transactions)

        # step4
        for tx in selected_transactions:
            tx.approved_by_transactions.append(transaction)
            transaction.approved_transactions.append(tx)

            # add the current transaction hash to the selected transaction header
            tx.previous_hashes.append(transaction.hash)

            # update tips list
            if tx in tipsList:
                tipsList.remove(tx)
                unconfirmedList.append(tx)

        # add the transaction to the transaction list of the current node
        self.entityTransactions.append(transaction)

        tipsList.append(transaction)
        print("\nTransaction is added to the tangle.")

        # step5
        transaction.update_cumulative_weights()

        # check if the cumulative weight of unconfirmed transactions reached the threshold
        for tx in unconfirmedList:
            if tx.cumulative_weight >= confirmed_threshold:
                unconfirmedList.remove(tx)
                confirmedList.append(tx)

        # print transaction details
        print(transaction)
        print("\nTip List: ", [tx.id for tx in tipsList])
        print("\nUnconfirmed List: ", [tx.id for tx in unconfirmedList])
        print("\nConfirmed List: ", [tx.id for tx in confirmedList])


    def transaction_selection(self, tipsList, unconfirmedList, num_txs):
        selected_transactions = tipsList[-num_txs:]
        num_selected_txs = len(selected_transactions)

        if num_selected_txs < num_txs:
            num_remaining = num_txs - num_selected_txs

            if len(unconfirmedList) >= num_remaining:
                selected_transactions.extend(unconfirmedList[-num_remaining:])
            else:
                selected_transactions.extend(unconfirmedList[:])

        return selected_transactions


    def validate_transactions(self, selected_transactions):
        new_selection = False

        for tx in selected_transactions:
            valid = True

            if not self.is_transaction_valid(tx):
                new_selection = True
                valid = False

                # create temp tip list and unconfirmed list to search for valid transactions
                temp_tipsList = [x for x in tipsList if x not in selected_transactions]
                temp_unconfirmedList = [x for x in unconfirmedList if x not in selected_transactions]

                print(f'\nTransaction {tx.id} is not valid. finding another transaction ...')

                while not valid:
                    new_tx = self.transaction_selection(temp_tipsList, temp_unconfirmedList, 1)

                    if not new_tx:
                        valid = True
                        print("\nThere is no other transaction to choose.")

                    elif self.is_transaction_valid(new_tx):
                        valid = True
                        selected_transactions.remove(tx)
                        selected_transactions.append(new_tx)

                    else: # if new transaction is also invalid
                        # remove the invalid transaction form tip/unconfirmed list not to choose it again
                        if new_tx in temp_tipsList:
                            temp_tipsList.remove(new_tx)
                        else:
                            temp_unconfirmedList.remove(new_tx)

        if new_selection:
            print("\nNewly selected transactions' IDs:")
            for tx in selected_transactions:
                print(tx.id)


    def is_transaction_valid(self, transaction):
        valid = True

        for tx in transaction.approved_transactions:
            if transaction.hash not in tx.previous_hashes:
                valid = False

        return valid


    def get_transactions(self):
        return ", ".join([f"{tx.id}" for tx in self.entityTransactions])

    def __str__(self):
        return f"Entity ({self.id})"


In [23]:
class Transaction:
    def __init__(self, id, timestamp, sender_entity):
        self.id = id
        self.timestamp = timestamp
        self.sender_entity = sender_entity

        self.data = self.get_random_string()
        self.weight = 1
        self.cumulative_weight = self.weight
        self.hash = None
        self.approved_transactions = []  # List of transactions approved by this transaction
        self.approved_by_transactions = []  # List of transactions that approve this transaction
        self.previous_hashes = []  # List of the hashes of the transactions that approved this transaction


    def get_random_string(self):
        return ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(5))


    def calculate_hash(self):
        block_hash = ""
        target = "0000"

        data_hash = hashlib.sha256(self.data.encode()).hexdigest()
        # Convert data_hash hexadecimal strings to bytes for concatination later on
        data_hash_bytes = bytes.fromhex(data_hash)

        while not block_hash.startswith(target):
          nonce = self.get_random_string()
          nonce_hash = hashlib.sha256(nonce.encode()).hexdigest()

          # Convert nonce_hash hexadecimal strings to bytes for concatination
          nonce_hash_bytes = bytes.fromhex(nonce_hash)

          # Concatenate the bytes
          concatenated_bytes = data_hash_bytes + nonce_hash_bytes
          # Hash the concatenated bytes using SHA-256
          block_hash = hashlib.sha256(concatenated_bytes).hexdigest()

        return block_hash


    def calculate_cumulative_weight(self):
        visited = set()
        total_weight = 0

        queue = deque([self])
        visited.add(self)

        while queue:
            current_transaction = queue.popleft()
            total_weight += current_transaction.weight

            for approved_transaction in current_transaction.approved_by_transactions:
                if approved_transaction not in visited:
                    visited.add(approved_transaction)
                    queue.append(approved_transaction)

        self.cumulative_weight = total_weight


    # update cumulative weights of transactions after adding a new transaction to the tangle
    def update_cumulative_weights(self):
        visited = set()
        for tx in self.approved_transactions:
            queue = deque([tx])
            visited.add(tx)

        while queue:
            current_transaction = queue.popleft()
            current_transaction.cumulative_weight += self.weight

            for tx in current_transaction.approved_transactions:
                if tx not in visited:
                    visited.add(tx)
                    queue.append(tx)


    def __str__(self):
        approved_transactions_str = ", ".join([f"{tx.id}" for tx in self.approved_transactions])

        return "\n*********************\nTransaction details\n*********************\n"\
               f"Transaction ID: {self.id}\n"\
               f"Transaction data: {self.data}\n"\
               f"Transaction weight: {self.weight}\n"\
               f"Transaction cumulative weight: {self.cumulative_weight}\n"\
               f"Approved Transactions: {approved_transactions_str}"


In [8]:
# prints cumulative weights of all transactions
def print_c_weights():
    tangle = confirmedList + unconfirmedList + tipsList
    c_weight_str = ""

    for tx in tangle:
        c_weight_str += f"Transaction ID: {tx.id}  Cumulative Weight: {tx.cumulative_weight}\n"

    print(c_weight_str)


In [28]:
tipsList = []  # Global FIFO list of tips
unconfirmedList = []  # Global list of unconfirmed transactions
confirmedList = []  # Global list of confirmed transactions
confirmed_threshold = 5  # threshold of cumulative weight for confirming a transaction

# genesis
genesis = Transaction(0, time.time(), 0)
tipsList.append(genesis)

# other nodes
node_1 = Entity(1)
node_2 = Entity(2)
node_3 = Entity(3)
node_4 = Entity(4)
node_5 = Entity(5)

In [ ]:
node_1.send_transaction()

Transaction hash:  00005317394faafaac66daf969750706164e0b82a968a71df77c15f8f8c04115

Selected transactions' IDs:
0

Transaction is added to the tangle.

*********************
Transaction details
*********************
Transaction ID: 48
Transaction data: 9xzcy
Transaction weight: 1
Transaction cumulative weight: 1
Approved Transactions: 0

Tip List:  [48]

Unconfirmed List:  [0]

Confirmed List:  []


In [ ]:
node_3.send_transaction()

Transaction hash:  0000711b05a5c764670a8e3d8ed38320ac4c4dfdba346793e80345f530286d65

Selected transactions' IDs:
48
0

Transaction is added to the tangle.

*********************
Transaction details
*********************
Transaction ID: 54
Transaction data: uzm31
Transaction weight: 1
Transaction cumulative weight: 1
Approved Transactions: 48, 0

Tip List:  [54]

Unconfirmed List:  [0, 48]

Confirmed List:  []


In [ ]:
node_5.send_transaction()

Transaction hash:  000098ba18bf8393b2b34fd1109d3180469ea003cbf51029971275a731bc4137

Selected transactions' IDs:
54
48

Transaction is added to the tangle.

*********************
Transaction details
*********************
Transaction ID: 6
Transaction data: 783x1
Transaction weight: 1
Transaction cumulative weight: 1
Approved Transactions: 54, 48

Tip List:  [6]

Unconfirmed List:  [0, 48, 54]

Confirmed List:  []


In [ ]:
node_2.send_transaction()

Transaction hash:  000029a2fd30059bdaa09b97f6c40617e2db444b76bf0e5cb1519e954d0b06e7

Selected transactions' IDs:
6
54

Transaction is added to the tangle.

*********************
Transaction details
*********************
Transaction ID: 1
Transaction data: ef3vd
Transaction weight: 1
Transaction cumulative weight: 1
Approved Transactions: 6, 54

Tip List:  [1]

Unconfirmed List:  [48, 54, 6]

Confirmed List:  [0]


In [ ]:
node_4.send_transaction()

Transaction hash:  00004b237670a9a6a4400f511588da40be543f355130cf951d380d1e791ac758

Selected transactions' IDs:
1
6

Transaction is added to the tangle.

*********************
Transaction details
*********************
Transaction ID: 57
Transaction data: shdqy
Transaction weight: 1
Transaction cumulative weight: 1
Approved Transactions: 1, 6

Tip List:  [57]

Unconfirmed List:  [48, 54, 6, 1]

Confirmed List:  [0]


In [ ]:
print_c_weights()

Transaction ID: 0  Cumulative Weight: 6
Transaction ID: 48  Cumulative Weight: 4
Transaction ID: 54  Cumulative Weight: 3
Transaction ID: 6  Cumulative Weight: 2
Transaction ID: 1  Cumulative Weight: 1
Transaction ID: 57  Cumulative Weight: 1



In [ ]:
node_5.send_transaction()

Transaction hash:  0000472ae05d273cddedf12ec65b59d07197185aa26686b40816e88f18bd84b7

Selected transactions' IDs:
57
1

Transaction is added to the tangle.

*********************
Transaction details
*********************
Transaction ID: 86
Transaction data: hwt93
Transaction weight: 1
Transaction cumulative weight: 1
Approved Transactions: 57, 1

Tip List:  [86]

Unconfirmed List:  [54, 6, 1, 57]

Confirmed List:  [0, 48]


In [ ]:
print_c_weights()

Transaction ID: 0  Cumulative Weight: 7
Transaction ID: 48  Cumulative Weight: 5
Transaction ID: 54  Cumulative Weight: 4
Transaction ID: 6  Cumulative Weight: 3
Transaction ID: 1  Cumulative Weight: 2
Transaction ID: 57  Cumulative Weight: 1
Transaction ID: 86  Cumulative Weight: 1

